<a href="https://colab.research.google.com/github/traderjoevitamins/librarian/blob/main/librarian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [61]:
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'
!pip install -U scikit-learn
!pip install tensorflow-text
!pip install bert-tensorflow
!pip install bert-for-tf2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=56e5361e13c471db5411996648f7f98b021414e6237935ac3f8e2aee1cf6db13
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19455 sha256=0efe228c50ea8cc32d243f3c3150463964414f8c328e0563c5df074c63205477
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7892 sha256=de0a8aef865d4f5f1da81cba388683082b47b7857c0a00db950cfe9453b38d5a
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e908768

In [62]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_data_validation as tfdv
import pandas as pd
import nltk
import tensorflow_hub as hub
import nltk
import sklearn
import bert
import math

from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from bert.tokenization import FullTokenizer
from bert import tokenization
#FullTokenizer = bert.bert_tokenization.FullTokenizer



nltk.download('punkt')
nltk.download('punkt')
print('TF version:', tf.__version__)
print('NLKT version:', nltk.__version__)
print('Hub version:', hub.__version__)
print('TFDV version:', tfdv.version.__version__)


TF version: 2.13.0
NLKT version: 3.8.1
Hub version: 0.14.0
TFDV version: 1.14.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [76]:
!pip install transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00


In [64]:
goodreads = pd.read_csv('/content/finalgoodreads.csv')
goodreads['enjoyed'] = 1
goodreads = goodreads[['Title', 'Author', 'ISBN13', 'synopsis', 'enjoyed']]
nyt = pd.read_csv('/content/nyt1.csv')
goodreads_synopsis = goodreads['synopsis']
nyt_synopsis = nyt['synopsis']

In [65]:
goodreads_synopsis = goodreads_synopsis.str.replace('<p class="null1">','').replace('</p>\n<p>', '')
goodreads_synopsis = goodreads_synopsis.str.replace('<p>', '').replace('<', '').replace('>', '')
goodreads_synopsis = goodreads_synopsis.str.replace('<BR>', '').replace('</p><h3>', '').replace('>', '')
goodreads_synopsis = str(goodreads_synopsis)


nyt_synopsis = nyt_synopsis.str.replace('<p class="null1">','').replace('</p>\n<p>', '')
nyt_synopsis = nyt_synopsis.str.replace('<p>', '').replace('<', '').replace('>', '')
nyt_synopsis = nyt_synopsis.str.replace('<BR>', '').replace('</p><h3>', '').replace('>', '')
nyt_synopsis = str(nyt_synopsis)

#goodreads_tokens = word_tokenize(goodreads_synopsis)
#nyt_tokens = word_tokenize(nyt_synopsis)

In [78]:
max_seq_length = len(nyt_synopsis)  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

In [79]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids


In [80]:
gtokens = tokenizer.tokenize(goodreads_synopsis)
ntokens = tokenizer.tokenize(nyt_synopsis)

gtokens = ["[CLS]"] + gtokens + ["[SEP]"]
ntokens = ["[CLS]"] + ntokens + ["[SEP]"]

ginput_ids = get_ids(gtokens, tokenizer, max_seq_length)
ninput_ids = get_ids(ntokens, tokenizer, max_seq_length)

ginput_masks = get_masks(gtokens, max_seq_length)
ninput_masks = get_masks(ntokens, max_seq_length)

ginput_segments = get_segments(gtokens, max_seq_length)
ninput_segments = get_segments(ntokens, max_seq_length)

pool_embs, all_embs = model.predict([[ginput_ids],[ginput_masks],[ginput_segments]])
#pool_embs, all_embs = model.predict([[ginput_ids],[ginput_masks],[ginput_segments]])


ValueError: ignored